In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("../")

In [2]:
import torch
import torch.nn.functional as F
from xent.tasks import Closure
from xent.models import M
from xent.lang import X
from xent.dataprocessing import Wikipedia
from xent.config import *

In [3]:
model = M("gpt2", "M0", base="base")
checker_model = M("gpt2", "M1-zero", base="closure")

In [4]:
corpus_generator = Wikipedia(split=0.8)
get_test_sample = corpus_generator.get_random_test_text

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [5]:
task = Closure(model)
synth = task.synthesize(get_test_sample, 1, out_type="string")

In [107]:
cut = task.find_xstring(synth[0], X.xreturn)
tasktokens = model.tokenize(synth[0]).input_ids.squeeze(0)

In [108]:
numbers = [" 0"," 1"," 2"," 3"," 4"," 5"," 6"," 7"," 8"," 9"," 10"," 11"," 12"," 13"," 14"," 15"," 16"," 17"," 18"," 19"," 20"," 21"," 22"," 23"," 24"," 25"," 26"," 27"," 28"," 29"," 30"]
numtoks = torch.tensor([model.tokenize(num).input_ids for num in numbers]).to(device)

In [109]:
CUT = cut+6
isnumber = torch.tensor([tok in numtoks for tok in tasktokens[CUT:]])
xentoks = tasktokens[CUT:][torch.where(isnumber)]

slices = [line for el in model.detokenize(tasktokens[CUT:]).split("\n\n") for line in el.split("\n")]

In [110]:
checker_model.model.eval()
with torch.no_grad():
    logits = checker_model.model(tasktokens.unsqueeze(0)).logits
    loss = F.cross_entropy(logits[0, CUT:-1], tasktokens.unsqueeze(0)[0, CUT+1:], reduction="none")
    # Get the predicted token probabilities
    probs = F.softmax(logits[0, CUT:-1], dim=-1)
    # Get the indices of tokens with highest probabilities
    highest_prob_tokens = torch.argmax(probs, dim=-1)
    # Convert to list for easier inspection

In [111]:
slices_2 = [line for el in model.detokenize(highest_prob_tokens).split("\n\n") for line in el.split("\n")]
xentok_isnum = torch.tensor([tok in numtoks for tok in highest_prob_tokens])
prob_xentoks = highest_prob_tokens[torch.where(xentok_isnum)]

In [112]:
model.tokenize(slices[42].split(":")[0]).input_ids in numtoks

True

In [115]:
skip_indices = []
for i, (s1, s2, x1, x2) in enumerate(zip(slices, slices_2, xentoks, prob_xentoks)):
    try: 
        if model.tokenize(s1.split(':')[0]).input_ids[0] in numtoks:
            skip_indices.append(i)
    except: pass
skip_indices = torch.tensor(skip_indices)
keep_mask = torch.ones(len(xentoks), dtype=torch.bool)
keep_mask[skip_indices] = False
xentoks = xentoks[torch.where(keep_mask)]
prob_xentoks = prob_xentoks[torch.where(keep_mask)]

In [116]:
skip_indices = []
for i, (s1, s2, x1, x2) in enumerate(zip(slices, slices_2, xentoks, prob_xentoks)):
    # Find the maximum length needed for both columns
    max_len_1 = max(len(str(s)) for s in slices[1:])
    max_len_2 = max(len(str(s)) for s in slices_2[1:])
    # Left-align s1 and s2 and pad with spaces
    s1_padded = f"{s1:<{max_len_1}}"
    s2_padded = f"{s2:<{max_len_2}}"
    print(f"{s1_padded} | {s2_padded} | {model.detokenize(x1):<5} | {model.detokenize(x2)} | {model.tokenize(s1.split(':')[0]).input_ids[0]}")


ione: 11            | : 11                |  11   |  11 | tensor([7935], device='cuda:2')
er: 0               | er: 0               |  0    |  0 | tensor([263], device='cuda:2')
 Cemetery: 12       |  Cemetery: 8        |  12   |  8 | tensor([35637], device='cuda:2')
 is: 3              |  is: 4              |  3    |  4 | tensor([318], device='cuda:2')
 a: 2               |  a: 2               |  2    |  2 | tensor([257], device='cuda:2')
 historic: 4        |  historic: 4        |  4    |  4 | tensor([9566], device='cuda:2')
 cemetery: 4        |  cemetery: 2        |  4    |  2 | tensor([27316], device='cuda:2')
 located: 2         |  located: 2         |  2    |  2 | tensor([5140], device='cuda:2')
 at: 2              |  at: 2              |  2    |  2 | tensor([379], device='cuda:2')
 Sidney: 9          |  Sidney: 8          |  9    |  8 | tensor([35727], device='cuda:2')
 in: 5              |  in: 4              |  5    |  4 | tensor([287], device='cuda:2')
 Delaware: 6        | 